In [1]:
import pandas as pd
import numpy as np

import re

from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize

In [2]:
train = pd.read_csv("input/train.csv")
test = pd.read_csv("input/test.csv")

df = pd.concat([train, test], axis = 0)

C:\Users\Pierce\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  after removing the cwd from sys.path.


In [3]:
#This dictionary came from Prashant Kikani's kernel in Kaggle, https://www.kaggle.com/prashantkikani/pooled-gru-with-preprocessing

repl = {
    "&lt;3": " good ",
    ":d": " good ",
    ":dd": " good ",
    ":p": " good ",
    "8)": " good ",
    ":-)": " good ",
    ":)": " good ",
    ";)": " good ",
    "(-:": " good ",
    "(:": " good ",
    "yay!": " good ",
    "yay": " good ",
    "yaay": " good ",
    "yaaay": " good ",
    "yaaaay": " good ",
    "yaaaaay": " good ",
    ":/": " bad ",
    ":&gt;": " sad ",
    ":')": " sad ",
    ":-(": " bad ",
    ":(": " bad ",
    ":s": " bad ",
    ":-s": " bad ",
    "&lt;3": " heart ",
    ":d": " smile ",
    ":p": " smile ",
    ":dd": " smile ",
    "8)": " smile ",
    ":-)": " smile ",
    ":)": " smile ",
    ";)": " smile ",
    "(-:": " smile ",
    "(:": " smile ",
    ":/": " worry ",
    ":&gt;": " angry ",
    ":')": " sad ",
    ":-(": " sad ",
    ":(": " sad ",
    ":s": " sad ",
    ":-s": " sad ",
    "r": "are",
    "u": "you",
    "haha": "ha",
    "hahaha": "ha",
    "don't": "do not",
    "doesn't": "does not",
    "didn't": "did not",
    "hasn't": "has not",
    "hasnt": "has not",
    "haven't": "have not",
    "hadn't": "had not",
    "won't": "will not",
    "wouldn't": "would not",
    "can't": "can not",
    "cannot": "can not",
    "i'm": "i am",
    "m": "am",
    "R" : "Are",
    "u": "you",
    "U" : "You",
    "haha": "ha",
    "hahaha": "ha",
    "don't": "do not",
    "doesn't": "does not",
    "didn't": "did not",
    "hasn't": "has not",
    "haven't": "have not",
    "hadn't": "had not",
    "won't": "will not",
    "wouldn't": "would not",
    "can't": "can not",
    "cannot": "can not",
    "couldnt": "could not",
    "i'm": "i am",
    "m": "am",
    "i'll" : "i will",
    "it's" : "it is",
    "'s" : " is",
    "that's" : "that is",
    "weren't" : "were not",
    "Y" : "why",
    "y" : "why",
    "EU" : "European Union",
    "US" : "United States",
    "U.S." : "United States",
    "ur" : "your"
}

df["comment_text"] = df["comment_text"].transform(lambda x: " ".join([repl[word] if word in repl else word for word in 
                          re.split(" ", x)]))

In [4]:
#word replacement
#Removing html tags
html_tags = re.compile("(http\S+|www.\S+)")
df["comment_text"] = df["comment_text"].str.replace(html_tags, "website")

#websites .com/.net/.org/.gov - prototype
website = re.compile("([A-Za-z0-9]\S+\.com|[A-Za-z0-9]\S+\.net|[A-Za-z0-9]\S+\.gov|[A-Za-z0-9]\S+\.org)")
df["comment_text"] = df["comment_text"].str.replace(website, "website")

#wikipedia - rpototype (usure if this feature would be useful on LSTM networks)
wiki = re.compile("wiki[a-zA-Z]\S*")
df["comment_text"] = df["comment_text"].str.replace(wiki, "wikipedia")

# email address - prototype
email = "[a-zA-Z0-9]+@[a-zA-Z]{2,}\.\S+\w"
df["comment_text"] = df["comment_text"].str.replace(email, "email")

In [5]:
#Noise Removal
#Removing IP Address
ip_address = re.compile("[0-9]{2,5}.[0-9]{2,5}.[0-9]{2,5}.[0-9]{2,5}")
df["comment_text"] = df["comment_text"].str.replace(ip_address, "")

#hashtag removal
hashtag = re.compile("#\S+")
df["comment_text"] = df["comment_text"].str.replace(hashtag, "")

#time stamp removal
timestamp = re.compile("([0-9]{2}:[0-9]{2}.+UTC\S+|[0-9]{2}:[0-9]{2}.*|[A-Za-z]{3,10}\s[0-9]{1,4}.*utc\S)", re.I)
df["comment_text"] = df["comment_text"].str.replace(timestamp, "")

#article id
article_id = re.compile("\d:\d\d\s{0,5}$")
df["comment_text"] = df["comment_text"].str.replace(article_id, "")

#username
username = re.compile("\[\[User(.*)\|")
df["comment_text"] = df["comment_text"].str.replace(username, "")

#added ==
equal = re.compile("==")
df["comment_text"] = df["comment_text"].str.replace(equal, "")

#filename extensions (jpeg, png) and miscellaneous
misc = re.compile("[A-Za-z]+:\S+")
df["comment_text"] = df["comment_text"].str.replace(misc, "")

#removing mutiple whitespaces
whitespace = re.compile("[ ]{2,}")
df["comment_text"] = df["comment_text"].str.replace(whitespace, " ")

#noise (alphabet characters followed by numeric character)- prototype
alpha_numeric = re.compile("[0-9]{5,}[a-zA-Z]{3,}\S+")
df["comment_text"] = df["comment_text"].str.replace(alpha_numeric, "")

#noise hold little information - parenthesis - prototype
paren = re.compile("\((.{,10})\)|[a-zA-Z0-9]+@\S+|\([0-9]{1,}\.[0-9]{1,}\.[0-9]{1,}\.[0-9]{1,}\)")
df["comment_text"] = df["comment_text"].str.replace(paren, "")

In [6]:
# single letter words

#dimensions
x_by = re.compile("(x|X)(?=\s[0-9]{1,})")
df["comment_text"] = df["comment_text"].str.replace(x_by, "by")

#single letter but not part of words in english dictionary
single = re.compile("\s[b-hj-zB-HJ-Z]\s")
df["comment_text"] = df["comment_text"].str.replace(single, "")

In [7]:
#separate words joind by (/, -)
# /
df["comment_text"] = df["comment_text"].str.split("(/)").transform(lambda x: 
                                                                        " ".join([re.sub("/", "or", y) for y in x]))

# - 
dash = re.compile("(-)|(_)")
df["comment_text"] = df["comment_text"].str.replace(dash, " ")

#for stastical methods (tf-idf)
# - removing punctuations
punctuation = re.compile("\W+")
df["comment_text"] = df["comment_text"].str.replace(punctuation, " ")

#normalize by lower case entire corpus
df["comment_text"] = df["comment_text"].str.lower()

In [8]:
#stem all words in the comments, which will be used for tf-idf
stemmer = SnowballStemmer("english")

df["comment_text"] = df["comment_text"].transform(lambda x: " ".join([stemmer.stem(word) for word in word_tokenize(x)]))

In [9]:
train = df.iloc[:len(train), :]
test = df.iloc[len(train):, :]

In [10]:
train.to_csv("train_clean.csv", index = False)
test.to_csv("test_clean.csv", index = False)